In [30]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb
import catboost as cb
from rgf.sklearn import RGFRegressor

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP
from dfdb import DFDB

import types
import copy

In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [3]:
pd.set_option('display.max_colwidth', -1)

In [4]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

In [5]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [6]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [7]:
original_columns = df_train.columns.drop(['index','y','label','group']).tolist()

In [8]:
common_columns50 = ["number_peaks{'n': 5}",
 'abs_q75_6',
 'q01_roll_std_100',
 '5000crest_factor_quantile75',
 'abs_q01_4',
 'q25_roll_std_100',
 'q05_roll_std_10',
 'median__roll_std',
 'abs_q05_6',
 '5000form_factor_quantile75',
 '5000smoothness_quantile05',
 "quantile{'q': 0.9}",
 'abs_q75_2',
 'q01_roll_std_1000',
 '5000quantile75mean_',
 'spkt_welch_density__coeff_3',
 "number_peaks{'n': 10}",
 "number_peaks{'n': 1}",
 '5000smoothness_mean_',
 'abs_q25_5',
 '5000std_quantile05',
 '5000smoothness_std_',
 '5000smoothness_median_',
 '5000median_variance_',
 'spkt_welch_density__coeff_4',
 '5000variance_quantile25',
 'abs_q95_2',
 'abs_q75_7',
 'q05_5',
 '5000smoothness_entropy_',
 '5000smoothness_quantile25',
 'q01_2',
 'q05_roll_std_100',
 '3th_peak_freq',
 "autocorrelation{'lag': 5}",
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'q01_roll_std_10',
 'abs_q01_3',
 '5000variance_median_',
 'q75_roll_std_10',
 '5000skewness_max_',
 'iqr_3',
 'abs_q01_5',
 'q75_roll_mean_10',
 '5000quantile99median_',
 'iqr_6',
 'Hilbert_mean_6',
 'q05_roll_std_1000',
 "number_peaks{'n': 3}",
 'spkt_welch_densitycoeff_2',
 '5000std_median_',
 '5000std_quantile25']

In [9]:
common_columns75 = ['abs_q75_6',
 'abs_q01_4',
 'q25_roll_std_100',
 'q05_roll_std_10',
 'median__roll_std',
 '5000smoothness_quantile05',
 'spkt_welch_density__coeff_3',
 "number_peaks{'n': 10}",
 'abs_q25_5',
 'abs_q75_7',
 'q05_5',
 'q05_roll_std_100',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'iqr_6',
 'q05_roll_std_1000']

In [10]:
common_columns95 = ['abs_q75_6',
 'abs_q01_4',
 'q25_roll_std_100',
 "number_peaks{'n': 10}",
 'abs_q25_5',
 'q05_roll_std_100',
 'iqr_6',
 'q05_roll_std_1000']

In [11]:
db = DFDB('../trial2/frgf.pkl', auto_commit=False)

In [22]:
param = {'columns': common_columns95,
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler'},
 'algorithm': {'cls': 'RGFRegressor',
  'init': {'max_leaf': 1805,
   'reg_depth': 64.1283408233688,
   'l2': 0.09393533729451725,
   'min_samples_leaf': 37,
   'learning_rate': 0.16112152084038367},
  'fit': {}},
}

In [23]:
# run one try
mytrial = []
df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, param, df_test = df_test, trial=mytrial, remark='try common_columns95')
db.insert(mytrial[0])
df_trial = db.select()

In [24]:
df_trial[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
0,2019-05-20 01:17:07.667428,52,1.984727,0.001383,2.108005,0.006824,0.123278,try common_columns50
1,2019-05-20 01:20:39.188667,15,2.029886,0.002838,2.077798,0.013472,0.047912,try common_columns75
2,2019-05-20 01:25:14.564817,8,2.047963,0.002703,2.086286,0.012955,0.038323,try common_columns95


In [25]:
db.commit()

In [26]:
mytrial = []
#  tune hypterparameters
def objective(trial):
        
    max_leaf = trial.suggest_int('max_leaf', 50, 5000)
    reg_depth = trial.suggest_uniform('reg_depth', 1.0, 100.0)
    l2 = trial.suggest_uniform('l2', 0.001, .1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 5, 100)
    learning_rate = trial.suggest_uniform('learning_rate', .05, .5)
        
    args={
        'columns':common_columns75,
        'kfold':{
            'n_splits': 3,
            'random_state': 1985,
            'shuffle': True,
            'type': 'group'
        },
        'scaler':{
            'cls':'StandardScaler',
        },
        'algorithm':{
            'cls':'RGFRegressor',
            'init':{
                "max_leaf":max_leaf,
                "reg_depth":reg_depth,
                "l2":l2,
                "min_samples_leaf":min_samples_leaf,
                "learning_rate":learning_rate,
            },
            'fit':{
            },
        }
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 1')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-05-20 01:29:13,766] Finished a trial resulted in value: 0.14066709099139202. Current best value is 0.14066709099139202 with parameters: {'max_leaf': 2793, 'reg_depth': 73.55099112679372, 'l2': 0.047858552095173715, 'min_samples_leaf': 66, 'learning_rate': 0.31120265058029567}.
[I 2019-05-20 01:30:20,122] Finished a trial resulted in value: 0.09718706888234101. Current best value is 0.09718706888234101 with parameters: {'max_leaf': 1641, 'reg_depth': 72.34390839280798, 'l2': 0.08586479660532954, 'min_samples_leaf': 76, 'learning_rate': 0.26502118595011703}.
[I 2019-05-20 01:31:59,575] Finished a trial resulted in value: 0.1990326037071719. Current best value is 0.09718706888234101 with parameters: {'max_leaf': 1641, 'reg_depth': 72.34390839280798, 'l2': 0.08586479660532954, 'min_samples_leaf': 76, 'learning_rate': 0.26502118595011703}.
[I 2019-05-20 01:36:20,969] Finished a trial resulted in value: 0.13844083955191003. Current best value is 0.09718706888234101 with parameters: {

[I 2019-05-20 02:15:33,304] Finished a trial resulted in value: 0.14942289689717517. Current best value is 0.04674998311602113 with parameters: {'max_leaf': 53, 'reg_depth': 36.33782586537499, 'l2': 0.07421443179183218, 'min_samples_leaf': 61, 'learning_rate': 0.3563143115134929}.
[I 2019-05-20 02:20:09,872] Finished a trial resulted in value: 0.26218337568662614. Current best value is 0.04674998311602113 with parameters: {'max_leaf': 53, 'reg_depth': 36.33782586537499, 'l2': 0.07421443179183218, 'min_samples_leaf': 61, 'learning_rate': 0.3563143115134929}.
[I 2019-05-20 02:21:31,461] Finished a trial resulted in value: 0.11403025425847887. Current best value is 0.04674998311602113 with parameters: {'max_leaf': 53, 'reg_depth': 36.33782586537499, 'l2': 0.07421443179183218, 'min_samples_leaf': 61, 'learning_rate': 0.3563143115134929}.
[I 2019-05-20 02:23:19,851] Finished a trial resulted in value: 0.12994175842548605. Current best value is 0.04674998311602113 with parameters: {'max_leaf

[I 2019-05-20 03:02:39,101] Finished a trial resulted in value: 0.17518640782783063. Current best value is 0.04112999988153636 with parameters: {'max_leaf': 82, 'reg_depth': 76.44461679899067, 'l2': 0.06733727044176047, 'min_samples_leaf': 8, 'learning_rate': 0.19670019409539508}.
[I 2019-05-20 03:03:21,896] Finished a trial resulted in value: 0.09357930267509629. Current best value is 0.04112999988153636 with parameters: {'max_leaf': 82, 'reg_depth': 76.44461679899067, 'l2': 0.06733727044176047, 'min_samples_leaf': 8, 'learning_rate': 0.19670019409539508}.
[I 2019-05-20 03:05:42,609] Finished a trial resulted in value: 0.17619809906028702. Current best value is 0.04112999988153636 with parameters: {'max_leaf': 82, 'reg_depth': 76.44461679899067, 'l2': 0.06733727044176047, 'min_samples_leaf': 8, 'learning_rate': 0.19670019409539508}.
[I 2019-05-20 03:07:04,096] Finished a trial resulted in value: 0.09695179212544362. Current best value is 0.04112999988153636 with parameters: {'max_leaf

[I 2019-05-20 03:26:43,087] Finished a trial resulted in value: 0.09025572955830267. Current best value is 0.03900511916998082 with parameters: {'max_leaf': 55, 'reg_depth': 82.27932192771073, 'l2': 0.05361794563201627, 'min_samples_leaf': 17, 'learning_rate': 0.11275425106770541}.
[I 2019-05-20 03:27:58,729] Finished a trial resulted in value: 0.1021214682464414. Current best value is 0.03900511916998082 with parameters: {'max_leaf': 55, 'reg_depth': 82.27932192771073, 'l2': 0.05361794563201627, 'min_samples_leaf': 17, 'learning_rate': 0.11275425106770541}.
[I 2019-05-20 03:28:43,660] Finished a trial resulted in value: 0.09897144119496154. Current best value is 0.03900511916998082 with parameters: {'max_leaf': 55, 'reg_depth': 82.27932192771073, 'l2': 0.05361794563201627, 'min_samples_leaf': 17, 'learning_rate': 0.11275425106770541}.
[I 2019-05-20 03:32:42,087] Finished a trial resulted in value: 0.1919221491901182. Current best value is 0.03900511916998082 with parameters: {'max_lea

[I 2019-05-20 03:59:32,379] Finished a trial resulted in value: 0.07046088283686985. Current best value is 0.03900511916998082 with parameters: {'max_leaf': 55, 'reg_depth': 82.27932192771073, 'l2': 0.05361794563201627, 'min_samples_leaf': 17, 'learning_rate': 0.11275425106770541}.
[I 2019-05-20 04:00:35,621] Finished a trial resulted in value: 0.1068910087507942. Current best value is 0.03900511916998082 with parameters: {'max_leaf': 55, 'reg_depth': 82.27932192771073, 'l2': 0.05361794563201627, 'min_samples_leaf': 17, 'learning_rate': 0.11275425106770541}.
[I 2019-05-20 04:01:55,641] Finished a trial resulted in value: 0.09998439557207585. Current best value is 0.03900511916998082 with parameters: {'max_leaf': 55, 'reg_depth': 82.27932192771073, 'l2': 0.05361794563201627, 'min_samples_leaf': 17, 'learning_rate': 0.11275425106770541}.
[I 2019-05-20 04:02:19,189] Finished a trial resulted in value: 0.09480024463518974. Current best value is 0.03900511916998082 with parameters: {'max_le

[I 2019-05-20 04:32:52,237] Finished a trial resulted in value: 0.1744201042867347. Current best value is 0.038563362015174595 with parameters: {'max_leaf': 50, 'reg_depth': 43.008034762821794, 'l2': 0.056472452181845496, 'min_samples_leaf': 44, 'learning_rate': 0.06602958371278184}.
[I 2019-05-20 04:33:36,378] Finished a trial resulted in value: 0.15722657443414084. Current best value is 0.038563362015174595 with parameters: {'max_leaf': 50, 'reg_depth': 43.008034762821794, 'l2': 0.056472452181845496, 'min_samples_leaf': 44, 'learning_rate': 0.06602958371278184}.
[I 2019-05-20 04:34:25,565] Finished a trial resulted in value: 0.15448698818519527. Current best value is 0.038563362015174595 with parameters: {'max_leaf': 50, 'reg_depth': 43.008034762821794, 'l2': 0.056472452181845496, 'min_samples_leaf': 44, 'learning_rate': 0.06602958371278184}.
[I 2019-05-20 04:34:47,938] Finished a trial resulted in value: 0.09637722579645426. Current best value is 0.038563362015174595 with parameters

[I 2019-05-20 04:48:01,744] Finished a trial resulted in value: 0.099856782515803. Current best value is 0.022974876740611522 with parameters: {'max_leaf': 54, 'reg_depth': 78.90619725873209, 'l2': 0.0570848242769325, 'min_samples_leaf': 32, 'learning_rate': 0.05015199252125539}.
[I 2019-05-20 04:49:09,959] Finished a trial resulted in value: 0.13031155503731567. Current best value is 0.022974876740611522 with parameters: {'max_leaf': 54, 'reg_depth': 78.90619725873209, 'l2': 0.0570848242769325, 'min_samples_leaf': 32, 'learning_rate': 0.05015199252125539}.
[I 2019-05-20 04:51:54,302] Finished a trial resulted in value: 0.1437191269072463. Current best value is 0.022974876740611522 with parameters: {'max_leaf': 54, 'reg_depth': 78.90619725873209, 'l2': 0.0570848242769325, 'min_samples_leaf': 32, 'learning_rate': 0.05015199252125539}.
[I 2019-05-20 04:52:57,231] Finished a trial resulted in value: 0.12245230123412731. Current best value is 0.022974876740611522 with parameters: {'max_lea

[I 2019-05-20 05:21:51,636] Finished a trial resulted in value: 0.10359519335312596. Current best value is 0.022974876740611522 with parameters: {'max_leaf': 54, 'reg_depth': 78.90619725873209, 'l2': 0.0570848242769325, 'min_samples_leaf': 32, 'learning_rate': 0.05015199252125539}.
[I 2019-05-20 05:21:58,852] Finished a trial resulted in value: 0.026827600326714007. Current best value is 0.022974876740611522 with parameters: {'max_leaf': 54, 'reg_depth': 78.90619725873209, 'l2': 0.0570848242769325, 'min_samples_leaf': 32, 'learning_rate': 0.05015199252125539}.


In [27]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()

In [28]:
df_trial[(df_trial['remark']=='tune 1')&(df_trial['mae_diff']<.05)][['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].sort_values(by=['val_mae']).head(10)

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
193,2019-05-20 05:14:12.435895,15,2.026708,0.002810,2.076501,0.013648,0.049793
90,2019-05-20 03:27:58.719074,15,2.027752,0.002815,2.076922,0.013569,0.049170
95,2019-05-20 03:35:12.396297,15,2.027125,0.002810,2.077015,0.013631,0.049890
119,2019-05-20 04:01:55.626863,15,2.029193,0.002831,2.077324,0.013475,0.048131
137,2019-05-20 04:23:10.250131,15,2.029197,0.002718,2.078052,0.013309,0.048855
192,2019-05-20 05:12:47.357673,15,2.030186,0.002838,2.078194,0.013448,0.048008
58,2019-05-20 02:57:41.847795,15,2.030906,0.002846,2.078299,0.013362,0.047393
201,2019-05-20 05:21:51.609971,15,2.028566,0.002780,2.078410,0.013632,0.049843
63,2019-05-20 03:07:04.088745,15,2.031876,0.002851,2.078520,0.013288,0.046645
4,2019-05-20 01:30:20.120727,15,2.032117,0.002840,2.078867,0.013277,0.046750


In [31]:
param = copy.deepcopy(df_trial.loc[193]['param'])
param['kfold']['type'] = 'stratified'
# run one try
mytrial=[]
df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, param, df_test = df_test, trial=mytrial, remark='remodel 193 use stratified')

In [32]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()

In [33]:
df_trial.tail(1)[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
203,2019-05-20 05:27:07.962205,15,2.035575,0.000003,2.048773,0.000069,0.013198


In [34]:
db.commit()

In [35]:
mytrial = []
#  tune hypterparameters
def objective(trial):
        
    max_leaf = trial.suggest_int('max_leaf', 50, 5000)
    reg_depth = trial.suggest_uniform('reg_depth', 1.0, 100.0)
    l2 = trial.suggest_uniform('l2', 0.001, .1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 5, 100)
    learning_rate = trial.suggest_uniform('learning_rate', .05, .5)
        
    args={
        'columns':common_columns75,
        'kfold':{
            'n_splits': 3,
            'random_state': 1985,
            'shuffle': True,
            'type': 'stratified'
        },
        'scaler':{
            'cls':'StandardScaler',
        },
        'algorithm':{
            'cls':'RGFRegressor',
            'init':{
                "max_leaf":max_leaf,
                "reg_depth":reg_depth,
                "l2":l2,
                "min_samples_leaf":min_samples_leaf,
                "learning_rate":learning_rate,
            },
            'fit':{
            },
        }
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 1 by stratified')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-05-20 05:34:27,232] Finished a trial resulted in value: 0.06832528993275716. Current best value is 0.06832528993275716 with parameters: {'max_leaf': 4805, 'reg_depth': 21.170830109515776, 'l2': 0.03305974760897526, 'min_samples_leaf': 77, 'learning_rate': 0.06662803745134985}.
[I 2019-05-20 05:34:49,143] Finished a trial resulted in value: 0.036880851638485. Current best value is 0.036880851638485 with parameters: {'max_leaf': 535, 'reg_depth': 16.140121481455804, 'l2': 0.030030926371857516, 'min_samples_leaf': 95, 'learning_rate': 0.2170199173248753}.
[I 2019-05-20 05:39:00,154] Finished a trial resulted in value: 0.03842386554643672. Current best value is 0.036880851638485 with parameters: {'max_leaf': 535, 'reg_depth': 16.140121481455804, 'l2': 0.030030926371857516, 'min_samples_leaf': 95, 'learning_rate': 0.2170199173248753}.
[I 2019-05-20 05:40:45,568] Finished a trial resulted in value: 0.05408379617527781. Current best value is 0.036880851638485 with parameters: {'max_le

[I 2019-05-20 06:31:58,509] Finished a trial resulted in value: 0.016684076016850638. Current best value is 0.016684076016850638 with parameters: {'max_leaf': 89, 'reg_depth': 33.84896413331784, 'l2': 0.09399445100757736, 'min_samples_leaf': 78, 'learning_rate': 0.14055695759050457}.
[I 2019-05-20 06:32:07,102] Finished a trial resulted in value: 0.030191672475145478. Current best value is 0.016684076016850638 with parameters: {'max_leaf': 89, 'reg_depth': 33.84896413331784, 'l2': 0.09399445100757736, 'min_samples_leaf': 78, 'learning_rate': 0.14055695759050457}.
[I 2019-05-20 06:32:37,530] Finished a trial resulted in value: 0.04378988404904414. Current best value is 0.016684076016850638 with parameters: {'max_leaf': 89, 'reg_depth': 33.84896413331784, 'l2': 0.09399445100757736, 'min_samples_leaf': 78, 'learning_rate': 0.14055695759050457}.
[I 2019-05-20 06:33:02,526] Finished a trial resulted in value: 0.027104183598715956. Current best value is 0.016684076016850638 with parameters: 

[I 2019-05-20 07:12:28,278] Finished a trial resulted in value: 0.015627228078189562. Current best value is 0.015627228078189562 with parameters: {'max_leaf': 109, 'reg_depth': 39.249210464887575, 'l2': 0.06929456172776845, 'min_samples_leaf': 71, 'learning_rate': 0.1390914835393143}.
[I 2019-05-20 07:16:49,557] Finished a trial resulted in value: 0.04269923123297245. Current best value is 0.015627228078189562 with parameters: {'max_leaf': 109, 'reg_depth': 39.249210464887575, 'l2': 0.06929456172776845, 'min_samples_leaf': 71, 'learning_rate': 0.1390914835393143}.
[I 2019-05-20 07:17:27,489] Finished a trial resulted in value: 0.0273143129044824. Current best value is 0.015627228078189562 with parameters: {'max_leaf': 109, 'reg_depth': 39.249210464887575, 'l2': 0.06929456172776845, 'min_samples_leaf': 71, 'learning_rate': 0.1390914835393143}.
[I 2019-05-20 07:17:45,967] Finished a trial resulted in value: 0.02182734229720634. Current best value is 0.015627228078189562 with parameters: 

OSError: [Errno 28] No space left on device
[W 2019-05-20 07:31:44,218] Setting trial status as TrialState.FAIL because of the following error: OSError(28, 'No space left on device')
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipython-input-35-b989d094c0c8>", line 36, in objective
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 1 by stratified')
  File "/home/ubuntu/jupyter/wangzhaoxu/ep/LANLEarthquakePrediction2019/common.py", line 148, in process
  File "/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/rgf/utils.py", line 662, in fit
    self._fit_regression_task(X, y, sample_weight, params)
  File "/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/rgf/rgf_model.py", line 375, in _fit_regressio

OSError: [Errno 28] No space left on device
[W 2019-05-20 07:31:44,555] Setting trial status as TrialState.FAIL because of the following error: OSError(28, 'No space left on device')
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipython-input-35-b989d094c0c8>", line 36, in objective
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 1 by stratified')
  File "/home/ubuntu/jupyter/wangzhaoxu/ep/LANLEarthquakePrediction2019/common.py", line 148, in process
  File "/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/rgf/utils.py", line 662, in fit
    self._fit_regression_task(X, y, sample_weight, params)
  File "/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/rgf/rgf_model.py", line 375, in _fit_regressio

...

OSError: [Errno 28] No space left on device
[W 2019-05-20 07:31:52,123] Setting trial status as TrialState.FAIL because of the following error: OSError(28, 'No space left on device')
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipython-input-35-b989d094c0c8>", line 36, in objective
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 1 by stratified')
  File "/home/ubuntu/jupyter/wangzhaoxu/ep/LANLEarthquakePrediction2019/common.py", line 148, in process
  File "/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/rgf/utils.py", line 662, in fit
    self._fit_regression_task(X, y, sample_weight, params)
  File "/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/rgf/rgf_model.py", line 375, in _fit_regressio

OSError: [Errno 28] No space left on device
[W 2019-05-20 07:31:52,573] Setting trial status as TrialState.FAIL because of the following error: OSError(28, 'No space left on device')
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipython-input-35-b989d094c0c8>", line 36, in objective
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 1 by stratified')
  File "/home/ubuntu/jupyter/wangzhaoxu/ep/LANLEarthquakePrediction2019/common.py", line 148, in process
  File "/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/rgf/utils.py", line 662, in fit
    self._fit_regression_task(X, y, sample_weight, params)
  File "/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/rgf/rgf_model.py", line 375, in _fit_regressio

OSError: [Errno 28] No space left on device
[W 2019-05-20 07:31:53,029] Setting trial status as TrialState.FAIL because of the following error: OSError(28, 'No space left on device')
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipython-input-35-b989d094c0c8>", line 36, in objective
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 1 by stratified')
  File "/home/ubuntu/jupyter/wangzhaoxu/ep/LANLEarthquakePrediction2019/common.py", line 148, in process
  File "/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/rgf/utils.py", line 662, in fit
    self._fit_regression_task(X, y, sample_weight, params)
  File "/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/rgf/rgf_model.py", line 375, in _fit_regressio

OSError: [Errno 28] No space left on device
[W 2019-05-20 07:31:53,486] Setting trial status as TrialState.FAIL because of the following error: OSError(28, 'No space left on device')
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipython-input-35-b989d094c0c8>", line 36, in objective
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 1 by stratified')
  File "/home/ubuntu/jupyter/wangzhaoxu/ep/LANLEarthquakePrediction2019/common.py", line 148, in process
  File "/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/rgf/utils.py", line 662, in fit
    self._fit_regression_task(X, y, sample_weight, params)
  File "/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/rgf/rgf_model.py", line 375, in _fit_regressio

OSError: [Errno 28] No space left on device
[W 2019-05-20 07:31:53,952] Setting trial status as TrialState.FAIL because of the following error: OSError(28, 'No space left on device')
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipython-input-35-b989d094c0c8>", line 36, in objective
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 1 by stratified')
  File "/home/ubuntu/jupyter/wangzhaoxu/ep/LANLEarthquakePrediction2019/common.py", line 148, in process
  File "/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/rgf/utils.py", line 662, in fit
    self._fit_regression_task(X, y, sample_weight, params)
  File "/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/rgf/rgf_model.py", line 375, in _fit_regressio

In [36]:
for trial_i in mytrial:
    db.insert(trial_i)

In [37]:
df_trial = db.select()
df_trial[(df_trial['remark']=='tune 1 by stratified')&(df_trial['mae_diff']<.05)].sort_values(by=['val_mae'])[['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].head(10)

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
252,2019-05-20 07:01:02.308361,tune 1 by stratified,15,1.963236,0.000025,1.992569,0.000085,0.029333
204,2019-05-20 05:34:27.231000,tune 1 by stratified,15,1.964527,0.000019,1.998712,0.000120,0.034185
207,2019-05-20 05:40:45.566529,tune 1 by stratified,15,1.973236,0.000019,2.000274,0.000099,0.027038
276,2019-05-20 07:31:13.986322,tune 1 by stratified,15,1.978990,0.000022,2.003441,0.000074,0.024451
248,2019-05-20 06:50:59.036837,tune 1 by stratified,15,1.977474,0.000022,2.005915,0.000113,0.028441
246,2019-05-20 06:47:44.154861,tune 1 by stratified,15,1.978439,0.000026,2.009448,0.000109,0.031009
267,2019-05-20 07:25:06.114589,tune 1 by stratified,15,1.981540,0.000024,2.010974,0.000117,0.029434
251,2019-05-20 06:59:38.685968,tune 1 by stratified,15,1.987852,0.000022,2.014719,0.000113,0.026867
239,2019-05-20 06:40:02.123580,tune 1 by stratified,15,1.991642,0.000026,2.017751,0.000110,0.026109
210,2019-05-20 05:45:53.379485,tune 1 by stratified,15,1.992512,0.000026,2.018108,0.000109,0.025597


In [38]:
param = copy.deepcopy(df_trial.loc[252]['param'])
param['kfold']['type'] = 'group'
# run one try
mytrial=[]
df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, param, df_test = df_test, trial=mytrial, remark='remodel 252 use group')

In [39]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()

In [40]:
df_trial.tail(1)[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
278,2019-05-20 08:01:29.325664,15,1.94251,0.001667,2.058223,0.012936,0.115712


In [41]:
db.commit()